# Try to predict score with unsupervised data (unlabeled data)

In [3]:
from sklearn.cluster import KMeans
import pandas as pd

data = pd.read

clusters = 10

kmeans = KMeans(n_clusters=clusters)
kmeans.fit()

SyntaxError: invalid syntax (Temp/ipykernel_16772/649866509.py, line 3)